In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from distutils.dir_util import copy_tree
from xlrd import open_workbook
from xlutils.copy import copy
import json

runs=30
rng = [0.8,1.2]
ndist=50000

rawdat={}
lcs={}
samples={}

pwsefflb=0.49875
pwseffub=0.5775

pwsemilb=0.456
pwsemiub=5.405

soecemilb=0.72
soecemiub=5.405

rawdat['pwsi']=3088.0
rawdat['pwseff']=0.525
rawdat['pwsemi']=1.08
rawdat['soeci']=1388
rawdat['soecemi']=1.08
rawdat['pemeci']=3800
rawdat['pemfci']=7339
rawdat['sofci']=7339
rawdat['ccsgi']=2626.0
rawdat['ccsci']=5252.0

lcs['pwsi']=[1001.15/3088,(1001.15/3088)]
lcs['pwsemi']=[0.72/1.08,(0.72/1.08)]
lcs['soeci']=[450.0/1388,(450.0/1388)]
lcs['soecemi']=[0.72/1.08,(0.72/1.08)]
lcs['pemeci']=[1500/3800.0,400.0/3800.0,(400.0/3800.0)]
lcs['pemfci']=[4000.0/7339,3000.0/7339,(3000.0/7339)]
lcs['sofci']=[4000.0/7339,3000.0/7339,(3000.0/7339)]
lcs['ccsgi']=[1422/2626.0,(1422/2626.0)]
lcs['ccsci']=[4091/5252.0,(4091/5252.0)]


In [2]:
for key in rawdat:
    mean=rawdat[key]
    if key=='pwseff':
        l1=pwsefflb
        l2=pwseffub
    elif key=='pwsemi':
        l1=pwsemilb
        l2=pwsemiub
    elif key=='soecemi':
        l1=soecemilb
        l2=soecemiub
    else:
        l1=0.8*mean
        l2=1.2*mean
    sigma=np.std([l1,mean,l2])
    dist=np.sort(np.random.normal(loc=mean, scale=sigma, size=ndist))
    dlen=len(dist)
    # filter
    for j in range(dlen):
        if dist[j]<l1 or dist[j]>l2:
            dist[j]=0
    dist=np.trim_zeros(dist)
    dlen=len(dist)
    idx=random.sample(range(dlen), runs)
    slst=[]
    for i in range(len(idx)):
        slst.append(dist[idx[i]])
    samples[key]=slst

# put model template in PWD, in folder titled "template"
path=os.getcwd()
print(path)
src=path+'\\template'
print(src)

D:\shared-vm\sa_scripts
D:\shared-vm\sa_scripts\template


In [3]:
#create empty folders and populate with template files

for i in range(1,runs+1):
    folname='\\sa'+str(i)
    newdir=path+folname
    os.mkdir(newdir)
    copy_tree(src,newdir)
    

In [4]:
cellpos={}
fnames={}
lcinc={}

cellpos['pwsi']='I56'
cellpos['pwseff']='G55'
cellpos['pwsemi']='F31'
cellpos['soeci']='I64'
cellpos['soecemi']='F48'
cellpos['pemeci']='I68'
cellpos['pemfci']='I46'
cellpos['sofci']='I51'
cellpos['ccsgi']='I38'
cellpos['ccsci']='I34'

fnames['pwsi']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['pwseff']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['pwsemi']='\\SubRES_TMPL\\SubRes_NewTechs_Trans.XLS'
fnames['soeci']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['soecemi']='\\BY_Trans.XLS'
fnames['pemeci']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['pemfci']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['sofci']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['ccsgi']='\\SubRES_TMPL\\SubRES_NewTechs.xls'
fnames['ccsci']='\\SubRES_TMPL\\SubRES_NewTechs.xls'

In [5]:
for obj in samples:
    for j in range(runs):
        wbloc=path+'\\sa'+str(j+1)+fnames[obj]
        rb = open_workbook(wbloc)
        wb = copy(rb)      
        s = wb.get_sheet(0)
        pos=cellpos[obj]
        r=int(pos[1:])-1
        c=ord(pos[0])-65
        s.write(r,c,samples[obj][j])
        if obj != "pwseff":
            for i in range(len(lcs[obj])):
                s.write(r+i+1,c,((lcs[obj][i])*samples[obj][j]))
        wb.save(wbloc)

In [ ]:
print(samples)

In [ ]:
with open('random_samples.json','w') as f:
    json.dump(samples,f)

with open('random_samples.json') as f:
    readtest = json.load(f)